# External APIs for Agents: Stock Price Agent

This notebook demonstrates **external API integration** for AI agents by building a stock price checker that:

1. **Fetches live stock prices** from Alpha Vantage API (free, similar to Frankfurter)
2. **Calculates investment costs** including broker fees and commissions
3. **Provides detailed breakdowns** with current market data
4. **Handles missing inputs** with clarifying questions

## Key Concepts Demonstrated

- **External API Integration**: Real-time market data from free public APIs
- **Agent Tool Development**: Custom tools for financial workflows
- **Error Handling**: Graceful degradation and rate limit management
- **Input Validation**: Missing parameter detection
- **Financial Calculations**: Multi-step cost computations

## Scenario
An investment assistant that helps users understand the total cost of stock purchases including current market prices, broker fees, and commissions.

**Note**: This demo uses **Alpha Vantage's free API** (just like the exercise uses Frankfurter for currency conversion). Both are free public APIs perfect for learning!

- **Demo**: Alpha Vantage for stock prices (Global Quote endpoint)
- **Exercise**: Frankfurter for currency exchange rates

Get your free Alpha Vantage API key at: https://www.alphavantage.co/support/#api-key

In [1]:
import os
import json
import requests
from datetime import datetime
from typing import Dict, Optional
from dataclasses import dataclass
from openai import OpenAI

# Initialize OpenAI client with Vocareum endpoint
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Environment ready!")

✅ Environment ready!


## Test Alpha Vantage API Connection

Let's verify the API is working before building the agent.

In [2]:
# Simple API Test
print("🧪 Testing Alpha Vantage API...")
print()

# Test with IBM stock (commonly used in Alpha Vantage examples)
test_url = "https://www.alphavantage.co/query"
test_params = {
    "function": "GLOBAL_QUOTE",
    "symbol": "IBM",
    "apikey": "demo"
}

try:
    response = requests.get(test_url, params=test_params, timeout=10)
    print(f"Status Code: {response.status_code}")
    print(f"Response URL: {response.url}")
    print()
    
    data = response.json()
    print("Raw API Response:")
    print(json.dumps(data, indent=2))
    print()
    
    # Check what we got
    if "Global Quote" in data:
        quote = data["Global Quote"]
        if "05. price" in quote:
            price = quote["05. price"]
            print(f"✅ API Working! IBM Price: ${price}")
        else:
            print("⚠️ API responded but no price data found")
            print(f"Available keys in Global Quote: {list(quote.keys())}")
    elif "Note" in data:
        print(f"⚠️ API Rate Limit: {data['Note']}")
    elif "Error Message" in data:
        print(f"❌ API Error: {data['Error Message']}")
    else:
        print(f"⚠️ Unexpected response format. Keys: {list(data.keys())}")
        
except Exception as e:
    print(f"❌ Error: {e}")

🧪 Testing Alpha Vantage API...

Status Code: 200
Response URL: https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=IBM&apikey=demo

Raw API Response:
{
  "Global Quote": {
    "01. symbol": "IBM",
    "02. open": "278.3800",
    "03. high": "285.4500",
    "04. low": "277.0000",
    "05. price": "280.7500",
    "06. volume": "3346753",
    "07. latest trading day": "2025-10-15",
    "08. previous close": "276.1500",
    "09. change": "4.6000",
    "10. change percent": "1.6658%"
  }
}

✅ API Working! IBM Price: $280.7500


## Define Data Models

In [3]:
@dataclass
class InvestmentRequest:
    """Data model for stock purchase request"""
    ticker: Optional[str] = None
    shares: Optional[int] = None
    broker_fee_pct: Optional[float] = None
    commission_per_trade: Optional[float] = None
    
    def get_missing_fields(self) -> list:
        """Return list of required fields that are missing"""
        required = []
        if self.ticker is None:
            required.append("ticker")
        if self.shares is None:
            required.append("shares")
        if self.broker_fee_pct is None:
            required.append("broker_fee_pct")
        if self.commission_per_trade is None:
            required.append("commission_per_trade")
        return required
    
    def is_complete(self) -> bool:
        """Check if all required fields are provided"""
        return len(self.get_missing_fields()) == 0

@dataclass
class InvestmentResult:
    """Data model for investment calculation results"""
    ticker: str
    shares: int
    current_price: float
    base_cost: float
    broker_fee: float
    commission: float
    total_cost: float
    broker_fee_pct: float
    commission_per_trade: float
    timestamp: str

print("✅ Data models defined!")

✅ Data models defined!


## Build the Stock Price Agent

In [4]:
class StockPriceAgent:
    """Stock investment agent with external API integration"""
    
    def __init__(self):
        # Using Alpha Vantage free API (similar to Frankfurter for currency)
        # Get your free API key at: https://www.alphavantage.co/support/#api-key
        self.api_key = "demo"  # Free demo key with limited requests
        pass
    
    def get_stock_price(self, ticker: str) -> Optional[float]:
        """
        Fetch stock price from Alpha Vantage API (free, similar to Frankfurter)
        
        This uses Alpha Vantage's Global Quote endpoint which provides:
        - Current price
        - Daily high/low
        - Volume
        - Previous close
        
        Args:
            ticker: Stock ticker symbol (e.g., 'AAPL', 'TSLA')
            
        Returns:
            Current stock price as float, or None if API call fails
        """
        try:
            # Alpha Vantage API - free tier available
            # Similar pattern to Frankfurter API used in the exercise
            url = "https://www.alphavantage.co/query"
            params = {
                "function": "GLOBAL_QUOTE",
                "symbol": ticker.upper(),
                "apikey": self.api_key
            }
            
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            
            data = response.json()
            
            # Check for API limit message
            if "Note" in data:
                print(f"⚠️ API rate limit reached. Using demo data for {ticker}")
                # Fallback to demo prices when API limit is hit
                demo_prices = {
                    "AAPL": 178.50,
                    "TSLA": 242.80,
                    "GOOGL": 142.30,
                    "MSFT": 378.90,
                    "NVDA": 495.20
                }
                return demo_prices.get(ticker.upper())
            
            # Extract price from Global Quote
            global_quote = data.get("Global Quote", {})
            price_str = global_quote.get("05. price")
            
            if price_str:
                price = float(price_str)
                print(f"📊 Fetched price for {ticker.upper()}: ${price:.2f}")
                return price
            else:
                print(f"⚠️ Price not found for ticker: {ticker}")
                return None
                
        except requests.exceptions.RequestException as e:
            print(f"⚠️ API request failed: {str(e)[:100]}")
            return None
        except (KeyError, ValueError, IndexError, TypeError) as e:
            print(f"⚠️ Error parsing API response: {str(e)[:100]}")
            return None
    
    def calculate_total_cost(self, ticker: str, shares: int, price: float,
                            broker_fee_pct: float, commission: float) -> InvestmentResult:
        """
        Calculate total investment cost including fees
        
        Args:
            ticker: Stock ticker symbol
            shares: Number of shares to purchase
            price: Current stock price
            broker_fee_pct: Broker fee percentage (e.g., 0.5 for 0.5%)
            commission: Flat commission per trade
            
        Returns:
            InvestmentResult with detailed breakdown
        """
        # Step 1: Calculate base cost
        base_cost = shares * price
        
        # Step 2: Calculate broker fee (percentage of base cost)
        broker_fee = base_cost * (broker_fee_pct / 100)
        
        # Step 3: Add flat commission
        # commission is already a flat amount
        
        # Step 4: Calculate total
        total_cost = base_cost + broker_fee + commission
        
        return InvestmentResult(
            ticker=ticker,
            shares=shares,
            current_price=price,
            base_cost=base_cost,
            broker_fee=broker_fee,
            commission=commission,
            total_cost=total_cost,
            broker_fee_pct=broker_fee_pct,
            commission_per_trade=commission,
            timestamp=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        )
    
    def extract_investment_info(self, user_input: str) -> InvestmentRequest:
        """
        Extract investment information from user input using LLM
        
        Args:
            user_input: Natural language investment request
            
        Returns:
            InvestmentRequest with extracted information
        """
        extraction_prompt = f"""Extract investment information from this user input: "{user_input}"

Extract and return ONLY the information in this exact JSON format:
{{
    "ticker": "stock ticker or null",
    "shares": number or null,
    "broker_fee_pct": number or null,
    "commission_per_trade": number or null
}}

Rules:
- For ticker: extract stock symbol (e.g., "Apple" → "AAPL", "Tesla" → "TSLA")
- For shares: extract number of shares (e.g., "10 shares" → 10)
- For broker_fee_pct: extract percentage (e.g., "0.5% fee" → 0.5)
- For commission_per_trade: extract dollar amount (e.g., "$5 commission" → 5)
- Use null for missing information
- Look for keywords: shares, broker fee, commission, flat fee

Return valid JSON only, no explanations.
"""
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": extraction_prompt}],
                temperature=0.1,
                max_tokens=200
            )
            
            content = response.choices[0].message.content.strip()
            
            # Clean up response
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0].strip()
            elif "```" in content:
                content = content.split("```")[1].strip()
            
            data = json.loads(content)
            
            return InvestmentRequest(
                ticker=data.get("ticker"),
                shares=data.get("shares"),
                broker_fee_pct=data.get("broker_fee_pct"),
                commission_per_trade=data.get("commission_per_trade")
            )
            
        except Exception as e:
            print(f"Error extracting investment info: {e}")
            return InvestmentRequest()
    
    def ask_clarifying_question(self, missing_fields: list) -> str:
        """
        Generate one clarifying question for missing information
        
        Args:
            missing_fields: List of missing required fields
            
        Returns:
            Natural language clarifying question
        """
        if not missing_fields:
            return ""
        
        # Prioritize most important missing field
        priority_order = ["ticker", "shares", "broker_fee_pct", "commission_per_trade"]
        
        for field in priority_order:
            if field in missing_fields:
                questions = {
                    "ticker": "What stock ticker do you want to buy (e.g., AAPL, GOOGL, TSLA)?",
                    "shares": "How many shares do you want to purchase?",
                    "broker_fee_pct": "What's your broker's fee percentage (use 0 if no percentage fee)?",
                    "commission_per_trade": "What's the flat commission per trade in dollars (use 0 if no commission)?"
                }
                return questions[field]
        
        return "Could you provide more details about your investment?"
    
    def format_result(self, result: InvestmentResult) -> str:
        """
        Format investment result into user-friendly breakdown
        
        Args:
            result: InvestmentResult object with calculation details
            
        Returns:
            Formatted string with itemized breakdown
        """
        return f"""📈 **Stock Investment Breakdown**

**Ticker**: {result.ticker}
**Current Price**: ${result.current_price:.2f} per share
**Shares**: {result.shares}

**Base Cost**: ${result.base_cost:.2f}
**Broker Fee ({result.broker_fee_pct}%)**: ${result.broker_fee:.2f}
**Commission**: ${result.commission:.2f}

**💰 Total Investment**: ${result.total_cost:.2f}

*Prices as of {result.timestamp}*"""
    
    def process_investment_request(self, user_input: str) -> str:
        """
        Main processing function for investment requests
        
        Args:
            user_input: Natural language investment request
            
        Returns:
            Either clarifying question or complete investment breakdown
        """
        # Extract investment information
        request = self.extract_investment_info(user_input)
        
        # Check for missing information
        missing_fields = request.get_missing_fields()
        if missing_fields:
            return self.ask_clarifying_question(missing_fields)
        
        # Fetch current stock price
        price = self.get_stock_price(request.ticker)
        if price is None:
            return f"❌ Unable to fetch current price for {request.ticker}. Please check the ticker symbol and try again."
        
        # Calculate total cost with fees
        result = self.calculate_total_cost(
            ticker=request.ticker,
            shares=request.shares,
            price=price,
            broker_fee_pct=request.broker_fee_pct,
            commission=request.commission_per_trade
        )
        
        # Return formatted breakdown
        return self.format_result(result)

print("✅ StockPriceAgent defined!")

✅ StockPriceAgent defined!


## Demo: Interactive Stock Investment Planning

In [5]:
# Initialize the Stock Price Agent
agent = StockPriceAgent()

def test_scenarios():
    """Test the agent with various investment scenarios"""
    
    test_cases = [
        "I want to buy 50 shares of MSFT with a 0.5% broker fee and $10 commission",
        "Buy 25 shares of IBM, broker charges 0.3% and $5 flat fee",
        "How much to invest in 100 shares of MSFT with 0% fees?",
        "IBM stock, 30 shares, need to know broker fee",
        "I want to buy some IBM shares"
    ]
    
    print("📊 STOCK INVESTMENT AGENT DEMO")
    print("="*60)
    print()
    
    for i, test_input in enumerate(test_cases, 1):
        print(f"📝 **Test Case {i}:**")
        print(f"User: {test_input}")
        print()
        
        response = agent.process_investment_request(test_input)
        print(f"🤖 Agent: {response}")
        print()
        print("-"*50)
        print()

# Run the demo
test_scenarios()

📊 STOCK INVESTMENT AGENT DEMO

📝 **Test Case 1:**
User: I want to buy 50 shares of MSFT with a 0.5% broker fee and $10 commission

📊 Fetched price for MSFT: $513.43
🤖 Agent: 📈 **Stock Investment Breakdown**

**Ticker**: MSFT
**Current Price**: $513.43 per share
**Shares**: 50

**Base Cost**: $25671.50
**Broker Fee (0.5%)**: $128.36
**Commission**: $10.00

**💰 Total Investment**: $25809.86

*Prices as of 2025-10-16 02:25:23*

--------------------------------------------------

📝 **Test Case 2:**
User: Buy 25 shares of IBM, broker charges 0.3% and $5 flat fee

📊 Fetched price for IBM: $280.75
🤖 Agent: 📈 **Stock Investment Breakdown**

**Ticker**: IBM
**Current Price**: $280.75 per share
**Shares**: 25

**Base Cost**: $7018.75
**Broker Fee (0.3%)**: $21.06
**Commission**: $5.00

**💰 Total Investment**: $7044.81

*Prices as of 2025-10-16 02:25:25*

--------------------------------------------------

📝 **Test Case 3:**
User: How much to invest in 100 shares of MSFT with 0% fees?

🤖 Agent: W

## Individual Tool Testing

In [6]:
print("🔧 INDIVIDUAL TOOL TESTING")
print("="*40)
print()

# Test 1: Stock Price API (Demo Data)
print("1️⃣ **Testing get_stock_price() tool:**")
test_tickers = ["IBM", "MSFT"]

for ticker in test_tickers:
    price = agent.get_stock_price(ticker)
    if price is None:
        print(f"   {ticker}: Price not available")

print()

# Test 2: Cost Calculation
print("2️⃣ **Testing calculate_total_cost() tool:**")
print("   Scenario: 50 shares of MSFT with 0.5% fee and $10 commission")

# Fetch price for calculation
msft_price = agent.get_stock_price("MSFT")
if msft_price:
    result = agent.calculate_total_cost(
        ticker="MSFT",
        shares=50,
        price=msft_price,
        broker_fee_pct=0.5,
        commission=10.0
    )
    
    print(f"   Base Cost: ${result.base_cost:.2f}")
    print(f"   Broker Fee: ${result.broker_fee:.2f}")
    print(f"   Commission: ${result.commission:.2f}")
    print(f"   Total: ${result.total_cost:.2f}")
else:
    print("   Could not fetch MSFT price for calculation test")

print()

# Test 3: Information Extraction
print("3️⃣ **Testing extract_investment_info() tool:**")
test_inputs = [
    "Buy 25 shares of IBM with 0.3% fee and $5 commission",
    "I want 100 shares of IBM",
    "MSFT stock, 30 shares"
]

for text in test_inputs:
    request = agent.extract_investment_info(text)
    print(f"   Input: '{text}'")
    print(f"   Extracted: ticker={request.ticker}, shares={request.shares}, fee={request.broker_fee_pct}%, commission=${request.commission_per_trade}")
    print()

🔧 INDIVIDUAL TOOL TESTING

1️⃣ **Testing get_stock_price() tool:**
📊 Fetched price for IBM: $280.75
📊 Fetched price for MSFT: $513.43

2️⃣ **Testing calculate_total_cost() tool:**
   Scenario: 50 shares of MSFT with 0.5% fee and $10 commission
📊 Fetched price for MSFT: $513.43
   Base Cost: $25671.50
   Broker Fee: $128.36
   Commission: $10.00
   Total: $25809.86

3️⃣ **Testing extract_investment_info() tool:**
   Input: 'Buy 25 shares of IBM with 0.3% fee and $5 commission'
   Extracted: ticker=IBM, shares=25, fee=0.3%, commission=$5

   Input: 'I want 100 shares of IBM'
   Extracted: ticker=IBM, shares=100, fee=None%, commission=$None

   Input: 'MSFT stock, 30 shares'
   Extracted: ticker=MSFT, shares=30, fee=None%, commission=$None



## Error Handling and Edge Cases

In [7]:
print("⚠️ ERROR HANDLING & EDGE CASES")
print("="*40)
print()

# Test Invalid Ticker
print("1️⃣ **Testing invalid ticker:**")
invalid_response = agent.process_investment_request("Buy 10 shares of INVALID with 0.5% fee and $5 commission")
print(f"   Response: {invalid_response}")
print()

# Test Missing Information
print("2️⃣ **Testing missing information:**")
incomplete_requests = [
    "I want to buy some stock",
    "Buy 25 shares of AAPL",
    "TSLA with 0.3% broker fee"
]

for req in incomplete_requests:
    print(f"   Input: '{req}'")
    response = agent.process_investment_request(req)
    print(f"   Response: {response}")
    print()

# Test Zero Fees
print("3️⃣ **Testing zero fees scenario:**")
msft_price = agent.get_stock_price("MSFT")
if msft_price:
    zero_fee_result = agent.calculate_total_cost(
        ticker="MSFT",
        shares=10,
        price=msft_price,
        broker_fee_pct=0.0,
        commission=0.0
    )
    print(f"   10 shares of MSFT at ${msft_price:.2f} with 0% fees = ${zero_fee_result.total_cost:.2f}")
else:
    print("   Could not fetch MSFT price")
print()

# Test High Fees
print("4️⃣ **Testing high fees scenario:**")
ibm_price = agent.get_stock_price("IBM")
if ibm_price:
    high_fee_result = agent.calculate_total_cost(
        ticker="IBM",
        shares=10,
        price=ibm_price,
        broker_fee_pct=2.0,
        commission=50.0
    )
    print(f"   10 shares of IBM at ${ibm_price:.2f} with 2% fee + $50 commission = ${high_fee_result.total_cost:.2f}")
else:
    print("   Could not fetch IBM price")

⚠️ ERROR HANDLING & EDGE CASES

1️⃣ **Testing invalid ticker:**
   Response: What stock ticker do you want to buy (e.g., AAPL, GOOGL, TSLA)?

2️⃣ **Testing missing information:**
   Input: 'I want to buy some stock'
   Response: What stock ticker do you want to buy (e.g., AAPL, GOOGL, TSLA)?

   Input: 'Buy 25 shares of AAPL'
   Response: What's your broker's fee percentage (use 0 if no percentage fee)?

   Input: 'TSLA with 0.3% broker fee'
   Response: How many shares do you want to purchase?

3️⃣ **Testing zero fees scenario:**
📊 Fetched price for MSFT: $513.43
   10 shares of MSFT at $513.43 with 0% fees = $5134.30

4️⃣ **Testing high fees scenario:**
📊 Fetched price for IBM: $280.75
   10 shares of IBM at $280.75 with 2% fee + $50 commission = $2913.65


## Key Learning Points

### 1. **External API Integration**
- **API Call Pattern**: Structure for making external HTTP requests
- **Error Handling**: Graceful degradation when data is unavailable
- **API Response Processing**: Extracting relevant information from responses
- **Demo vs Production**: This demo uses sample data for reliability; see exercise for working API example

### 2. **Agent Tool Development**
- **Tool Composition**: Multiple specialized tools working together
- **Tool Orchestration**: Coordinated workflow across data fetching and calculations
- **Error Recovery**: Handling individual tool failures without breaking the workflow

### 3. **Input Processing**
- **Natural Language Extraction**: LLM-powered information extraction
- **Missing Data Handling**: Intelligent clarifying questions
- **Data Validation**: Type checking and format validation

### 4. **Financial Calculations**
- **Multi-step Computations**: Base cost, percentage fees, flat commissions
- **Transparent Breakdowns**: Clear itemization of all components
- **Accurate Arithmetic**: Proper calculation order

### 5. **Production Considerations**
- **API Authentication**: Managing API keys securely
- **Rate Limiting**: Respecting API usage limits
- **Caching Strategies**: Reducing redundant API calls
- **Fallback Mechanisms**: Alternative data sources
- **Audit Trails**: Timestamped results for record keeping

### 6. **Real API Example**
**The exercise (FX Rate Agent) provides a fully functional external API implementation** using:
- **Frankfurter API**: Free currency conversion API
- **Real HTTP requests**: Using `requests.get()`
- **JSON parsing**: Extracting exchange rates
- **Error handling**: Managing network failures

This pattern extends to other financial APIs like:
- **Cryptocurrency prices** (CoinGecko API)
- **Bond yields** (FRED API)
- **Options pricing** (Options Pricing APIs)
- **Economic indicators** (World Bank API)